In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

In [3]:
# Load the model , scaler, onehot
model = load_model('model.h5')

# load the scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# load the onehot encoder
with open('one_hot_encoding_geography.pkl', 'rb') as f:
    one_hot_encoding_geography = pickle.load(f)

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)


In [4]:
# Sample input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
# OneHot Encoding and label encoding
geography_encoded = one_hot_encoding_geography.transform([input_df['Geography']])
input_df['Gender'] = label_encoder_gender.transform([input_df['Gender']])
input_df


d:\ML\ANN Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
d:\ML\ANN Classification\venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [7]:
input_df_final = pd.DataFrame(geography_encoded.toarray(), columns=one_hot_encoding_geography.get_feature_names_out(['Geography']))
input_df = pd.concat([input_df, input_df_final], axis=1)
input_df.drop(columns=['Geography'], axis=1, inplace=True)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
# scaling the input data
input_df_scaled = scaler.transform(input_df)
input_df_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [9]:
# prediction
prediction = model.predict(input_df_scaled)
print("Prediction (Probability of Churn):", prediction[0][0])

1/1 [==============================] - 1s 773ms/step
Prediction (Probability of Churn): 0.038001575


In [10]:
if prediction >0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
